# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


<b>Описание данных</b>
<ul>
    <li>children — количество детей в семье</li>
    <li>days_employed — общий трудовой стаж в днях</li>
    <li>dob_years — возраст клиента в годах</li>
    <li>education — уровень образования клиента</li>
    <li>education_id — идентификатор уровня образования</li>
    <li>family_status — семейное положение</li>
    <li>family_status_id — идентификатор семейного положения</li>
    <li>gender — пол клиента</li>
    <li>income_type — тип занятости</li>
    <li>debt — имел ли задолженность по возврату кредитов</li>
    <li>total_income — ежемесячный доход</li>
    <li>purpose — цель получения кредита</li>
</ul>

<b>Задачи исследования</b>
<ol>
    <li>Просмотр данных</li>
    <ol>
        <li>посмотреть общую информацию о датасете</li>
        <li>посмотреть информацию в начале и в конце датасета</li>
        <li>сделать предварительные выводы</li>
        <li>найти аномалии</li>
    </ol>    
    <li>Обработка данных</li>
    <ol>
        <li>найти и обработать пропуски</li>
        <li>заменить некорректные типы данных</li>
        <li>удалить дубликаты</li>
        <li>выделить леммы</li>
        <li>Категоризировать данные</li>
    </ol> 
    <li>Ответы на вопросы</li>
    <ol>
        <li>Есть ли зависимость между наличием детей и возвратом кредита в срок?</li>
        <li>Есть ли зависимость между семейным положением и возвратом кредита в срок?</li>
        <li>Есть ли зависимость между уровнем дохода и возвратом кредита в срок?</li>
        <li>Как разные цели кредита влияют на его возврат в срок?</li>
    </ol> 
    <li>Итоговые результаты исследования</li>
    <ol>
        <li>Выводы, рекомендации</li>
    </ol> 
</ol>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd #импортирую библиотеку
df = pd.read_csv ('/datasets/data.csv') #читаю данные датасета
df.head(10)#просматриваю первые 10 строк датасета

from pymystem3 import Mystem
from collections import Counter

In [2]:
df.head(10)#просматриваю первые 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
df.tail(10)#просматриваю последние 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [4]:
df.describe()#просматриваю описательную статистику датасета

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<b>Наблюдения</b>
<ul>
    <li>В столбце <b>"children"</b> есть минимальное значение -1 (такого быть не может, а значит - ошибка). Предположу, что должно быть просто 1</li>
    <li>Максимальное значение в столбце <b>"children"</b> - 20. Необходимо проверить, сколько всего таких строк имеется в датафрейме. Возможно здесь ошибка.</li>
    <li>Отрицательные значения по количеству отработанных дней <b>"days_employed"</b>. Пока неочень понятно как так, но может нужно взять просто модуль</li>
    <li>Среднее значение по количеству отработанных дней <b>"days_employed"</b> - 63046 дней (~173 года). Такого просто не может быть</li>
    <li>По некоторым строкам возраст <b>"dob_years"</b> равен 0. Возможно, эти данные были пропущены, но при оформлении кредита обязательно должны заполнять такие поля</li>
</ul>

In [5]:
df.info() #просматриваю общую информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b>Наблюдения</b>
<ul>
    <li>Отсутствуют значения только в столбцах <b>"days_employed"</b> и <b>"debt"</b>.</li>
    <li>Столбцам <b>"days_employed"</b> и <b>"debt"</b> стоит присвоить целочисленные значения. Так будет удобнее работать</li>
</ul>

## Шаг 2. Предобработка данных

Переименую столбцы, чтобы названия колонок были понятнее и отображали суть

In [6]:
df = df.rename(columns={'children' : 'number_of_children', 'dob_years' : 'client_age_in_years', 'debt':'credit_status', 'income_type':'employment_type', 'total_income':'monthly_income', 'purpose' : 'credit_purpose'})

In [7]:
df.columns #проверяю, что столбцы переименованы

Index(['number_of_children', 'days_employed', 'client_age_in_years',
       'education', 'education_id', 'family_status', 'family_status_id',
       'gender', 'employment_type', 'credit_status', 'monthly_income',
       'credit_purpose'],
      dtype='object')

### Обработка пропусков

In [8]:
#посчитаем сумму по всем пропускам
df.isnull().sum()

number_of_children        0
days_employed          2174
client_age_in_years       0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
employment_type           0
credit_status             0
monthly_income         2174
credit_purpose            0
dtype: int64

**Вывод**
Одинаковое количество пропусков в днях работы и в доходе в месяц. Возможно, произошла ошибка при выгрузке данных, либо эти люди никогда не работали.

In [9]:
# Заменим пропуски на нули, так как при подстановке среднего значения может повыситься шанс на кредит
df['days_employed'] = df['days_employed'].fillna(0) 
df['monthly_income'] = df['monthly_income'].fillna(0)

In [10]:
df.isnull().sum()# Проверим замену

number_of_children     0
days_employed          0
client_age_in_years    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
employment_type        0
credit_status          0
monthly_income         0
credit_purpose         0
dtype: int64

In [11]:
# Разберемся с образованием
len(df['education'].unique())

15

In [12]:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

Видим, что уровней образования 15, а id всего 5. Посмотрим, что не так

In [13]:
df['education'].unique()#получаем набор уникальных значений в столбце education

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Теперь всё понятно. Значения записаны в разном регистре. Исправим это

In [14]:
# Значения из столбца "образование" запишем в едином стиле (в low-индекс)
df['education'] = df['education'].str.lower()

In [15]:
df['education'].unique()#проверяем набор уникальных значений в столбце education

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь количество уникальных значений соответствует количеству id

In [16]:
# Посмотрим что с детьми
df['number_of_children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

-1 и 20 явно выбиваются из остальных данных. Нужно это проверить

In [17]:
# Подсчитаем кол-во строк со значением -1
df[df['number_of_children'] == -1]['number_of_children'].count()

47

In [18]:
# Заменим -1 на 1
df['number_of_children'] = df['number_of_children'].replace(-1, 1)

In [19]:
# Проверим, сработала ли замена. Снова подсчитаем кол-во строк со значением -1
df[df['number_of_children'] == -1]['number_of_children'].count()

0

Отлично. Замена -1 сработала. Теперь необходимо разобраться со значение 20 (может быть такое действительно возможно)

In [20]:
# Подсчитаем количество строк с 20 детьми
df[df['number_of_children'] == 20]['number_of_children'].count()

76

Да, в данных явно есть какая-то ошибка. Возможно тех. ошибка при выгрузке данных

In [21]:
#сгруппируем по возрасту, чтобы убедиться в корректности данных
df[df['number_of_children'] == 20].groupby('client_age_in_years')['client_age_in_years'].count()

client_age_in_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: client_age_in_years, dtype: int64

In [22]:
df['number_of_children'].value_counts()#Найдем все уникальные значения столбца number_of_children и их кол-ва

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: number_of_children, dtype: int64

Выглядит всё таки странно. Предполагаю, что это ошибка, и должно быть значение 2. Заменяю 20 на 2.

In [23]:
df['number_of_children'] = df['number_of_children'].replace(20, 2)

In [24]:
#проверяем
df['number_of_children'].unique()

array([1, 0, 3, 2, 4, 5])

Теперь проверю столбец с семейным статусом **family_status**

In [25]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [26]:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Количество уникальных значений семейного статуса и id совпадают. Но в столбце family_status необходимо перевести данные в нижний регистр

In [27]:
# Значения из столбца "семейный статус" запишем в едином стиле (в low-индекс)
df['family_status'] = df['family_status'].str.lower()

In [28]:
#проверяем
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [29]:
#Теперь посмотрим, что с полом
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Кроме женского и мужского есть что-то непонятное XNA

In [30]:
#Посчитаем кол-во строк, в которых есть этот странный пол
df[df['gender'] == 'XNA'].count()

number_of_children     1
days_employed          1
client_age_in_years    1
education              1
education_id           1
family_status          1
family_status_id       1
gender                 1
employment_type        1
credit_status          1
monthly_income         1
credit_purpose         1
dtype: int64

Возможно, человек действительно относит себя к такому полу. Оставлю, как есть и не буду ничего заменять (толерантность :) ). Это никак не повлияет на результат исследования.

In [31]:
#Посмотрим на столбец с типом занятости "employment_type"
df['employment_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Тут, вроде, всё хорошо

In [32]:
# Разберемся с возрастом
df['client_age_in_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: client_age_in_years, dtype: float64

In [33]:
# смотрим количество нулей по возрасту (столбец 'dob_years') и количество несовершеннолетних
print("Количество строк с возрастом 0 -", df[df['client_age_in_years'] == 0].count()[0])
print("Количество строк с возрастом < 18 -", df[df['client_age_in_years'] < 18].count()[0])

Количество строк с возрастом 0 - 101
Количество строк с возрастом < 18 - 101


Количество совпадает, следовательно, это одни и те же люди. Заменим 0 на среднее значение для каждой группы (по типу занятости)

In [34]:
df['client_age_in_years'] = df.groupby('employment_type')['client_age_in_years'].transform(lambda x: x.replace(0, int(x.mean())))

In [35]:
#проверим, что замена прошла успешно
print("Количество строк с возрастом 0 -", df[df['client_age_in_years'] == 0].count()[0])

Количество строк с возрастом 0 - 0


Теперь необходимо разобраться со столбцом **days_employed** (трудовой стаж в днях).

In [36]:
df['days_employed'].describe()

count     21525.000000
mean      56678.874622
std      134870.763085
min      -18388.949901
25%       -2518.168900
50%        -982.531720
75%           0.000000
max      401755.400475
Name: days_employed, dtype: float64

In [37]:
#для начала стоит разобраться с отрицательными значениями
df['days_employed'] = df['days_employed'].abs()

In [38]:
df['days_employed'].describe()#проверяем

count     21525.000000
mean      60156.419005
std      133355.929525
min           0.000000
25%         610.652074
50%        1808.053434
75%        4779.587738
max      401755.400475
Name: days_employed, dtype: float64

Теперь удобнее работать с данными. Значение максимум смущает. Слишком большой стаж. Необходимо проверить.

In [39]:
#считаю, что официальное трудоустройство разрешено с 16 лет
#нахожу самого старшего человека в наборе
max_days_of_employment = (df['client_age_in_years'].max() - 16) * 365
max_days_of_employment # Получим количество дней, по которым можно сделать отсев

21535

Самый великовозрастный человек проработал 21535 дней. Это значение на порядок меньше максимального в столбце. Найду долю таких странных максимальных значений в столбце "days_employed"

In [40]:
df[df['days_employed'] > max_days_of_employment]['days_employed'].count() / df['days_employed'].count() * 100

16.004645760743323

16% от всего набора. Это немало. Переведу значения в года и добавлю новый столбец. Так будет убодней работать, и вряд ли люди знают, сколько дней они проработали. В годах считать удобней

In [41]:
df['years_employed'] = df['days_employed'] / 365

In [42]:
df.head(15)

,number_of_children,days_employed,client_age_in_years,education,education_id,family_status,family_status_id,gender,employment_type,credit_status,monthly_income,credit_purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,2.537495
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7.888225
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,0.418574
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,18.985932
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,5.996593


### Замена типа данных

Используем функцию astype, т.к. столбцы у нас имеют чистый формат 'float64', то есть перевод в 'int' будет безопасным.

In [43]:
#Месячную зарплату monthly_income, стаж в днях days_employed и стаж в годах years_employed переводим в int.
df['days_employed'] = df['days_employed'].astype('int')
df['years_employed'] = df['years_employed'].astype('int')
df['monthly_income'] = df['monthly_income'].astype('int')

In [44]:
df.dtypes #выводим информацию о типах данных в таблице

number_of_children      int64
days_employed           int64
client_age_in_years     int64
education              object
education_id            int64
family_status          object
family_status_id        int64
gender                 object
employment_type        object
credit_status           int64
monthly_income          int64
credit_purpose         object
years_employed          int64
dtype: object

**Вывод** Все данные датасета представлены в удобном формате. Месячную зарплату, стаж в днях и стаж в годах перевели в 'int', чтобы видеть целочисленные значения. Остальные типы данных везде выглядят хорошо

### Обработка дубликатов

Используем стандартный метод "duplicated()", выводим суммарное количество дубликатов по датафрейму

In [45]:
df.duplicated().sum()

71

In [46]:
# Удалим дубли и сбросим индекс
df = df.drop_duplicates().reset_index(drop=True)

In [47]:
# Проверим удалились ли дубли
df.duplicated().sum()

0

In [48]:
df.head(15)

,number_of_children,days_employed,client_age_in_years,education,education_id,family_status,family_status_id,gender,employment_type,credit_status,monthly_income,credit_purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


**Вывод** Очистили датафрейм от дубликатов. Возможно, банку стоит проверить валидацию на сервере. Там заявки должны проверяться на дубли автоматически

### Лемматизация

В датасете есть столбец credit_purpose, в котором указаны цели на кредит. Необходимо выяснить количество уникальных лемм

In [49]:
# Узнаем уникальные значения
unique_credit_purposes = df['credit_purpose'].unique().tolist()
unique_credit_purposes

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Видим, что есть похожие друг на друга цели, но выраженные разными словами. Лемматизируем полученный выше список, получив "рейтинг" уникальных лемм

In [50]:
m = Mystem()
list_of_lemmas = []
for element in df['credit_purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)#отсортирую список лемм по убыванию "рейтинга"(кол-во повторений)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

<b>Выделим основные цели кредита:</b>
<ul>
    <li>Недвижимость</li>
    <li>Автомобиль</li>
    <li>Образование</li>
    <li>Свадьба</li>
</ul>

**Вывод** выделили леммы и получили 4 основные цели, на которые люди берут кредит чаще всего

### Категоризация данных

Благодаря лемматизации мы выявили 4 основные цели на получение кредита. Теперь использую метод apply() объединим данные в категории в столбце credit_purpose (цели кредитования). Это сократит объём данных и упростит их чтение.

In [51]:
def credit_purpose_category (row):
    credit_purpose = row['credit_purpose']
    if 'недв' in row['credit_purpose']:
        return 'недвижимость'
    if 'жил' in row['credit_purpose']:
        return 'недвижимость'
    if 'строит' in row['credit_purpose']:
        return 'недвижимость'
    if 'образ' in row['credit_purpose']:
        return 'образование'
    if 'авто' in row['credit_purpose']:
        return 'автомобиль'
    if 'свад' in row['credit_purpose']:
        return 'свадьба'
    else:
        return 'потребительский'
 
df['credit_purpose'] = df.apply(credit_purpose_category, axis=1)#заменяем старые данные в столбце credit_purpose новыми, применяем метод к строкам
print (df['credit_purpose'].value_counts())#узнаем, сколько записей разных категорий в столбце credit_purpose

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: credit_purpose, dtype: int64


Объединим данные в категории в столбце number_of_children (количество детей)

In [52]:
def children_category(row):
    if row['number_of_children'] == 0:
        return 'детей нет'
    if 1 <= row['number_of_children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'
df['number_of_children'] = df.apply(children_category, axis=1)#заменяем старые данные в столбце number_of_children новыми
print (df['number_of_children'].value_counts())#узнаем, сколько записей разных категорий в столбце number_of_children

детей нет      14091
1-2 ребенка     6983
многодетные      380
Name: number_of_children, dtype: int64


Объединим данные в категории в столбце years_employed (рабочий стаж в годах)

In [53]:
def years_employed_category(row):
    if row['years_employed'] < 10:
        return 'стаж до 10 лет'
    if 10 <= row['years_employed'] <= 30:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'
df['years_employed'] = df.apply(years_employed_category, axis=1)#заменяем старые данные в столбце years_employed новыми
print (df['years_employed'].value_counts())#узнаем, сколько записей разных категорий в столбце years_employed

стаж до 10 лет       14860
стаж более 30 лет     3589
стаж 10-30 лет        3005
Name: years_employed, dtype: int64


Также считаю, что нужно объединить данные в категории по уровню дохода. Но так как мы не знаем по какой стране/городу предоставлены данные для анализа, то разделим доход на 4 группы

In [54]:
df['monthly_income'].quantile([0.25,0.5,0.75]) #выделим группы доходов по квантилям

0.25     89088.50
0.50    135781.00
0.75    195813.25
Name: monthly_income, dtype: float64

In [55]:
def income_status_category(monthly_income):
    if monthly_income <= df['monthly_income'].quantile(0.25):
            return 'Низкий уровень дохода'
    if monthly_income <= df['monthly_income'].quantile(0.50):
            return 'Средний уровень дохода'
    if monthly_income < df['monthly_income'].quantile(0.75):
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'
df['income_status'] = df['monthly_income'].apply(income_status_category)

In [56]:
df.head(15)#смотрим, как изминились данные

,number_of_children,days_employed,client_age_in_years,education,education_id,family_status,family_status_id,gender,employment_type,credit_status,monthly_income,credit_purpose,years_employed,income_status
0,1-2 ребенка,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,стаж 10-30 лет,Сверхвысокий уровень дохода
1,1-2 ребенка,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,стаж 10-30 лет,Средний уровень дохода
2,детей нет,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,стаж 10-30 лет,Высокий уровень дохода
3,многодетные,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,стаж 10-30 лет,Сверхвысокий уровень дохода
4,детей нет,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,стаж более 30 лет,Высокий уровень дохода
5,детей нет,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,стаж до 10 лет,Сверхвысокий уровень дохода
6,детей нет,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,стаж до 10 лет,Сверхвысокий уровень дохода
7,детей нет,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,стаж до 10 лет,Высокий уровень дохода
8,1-2 ребенка,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,стаж 10-30 лет,Средний уровень дохода
9,детей нет,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,стаж до 10 лет,Высокий уровень дохода


**Вывод**
С помощью категоризации мы сократили объём данных и упростили их чтение.
Были выделены категории для уровня дохода, стажа работы, целей получения кредита и количества детей

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на данный вопрос подойдёт сводная таблица. По ней легко будет ответить и дать краткие пояснения

In [57]:
#произведем группировку данных по столбцу number_of_children
#группировку значений будем производить по слобцу credit_status
#значения, по которым мы хотим увидеть сводную таблицу берем из credit_purpose
#функцией count будем производить подсчет количества значений
children_pivot = df.pivot_table(index = ['number_of_children'], 
                                columns = ['credit_status'],
                                values = 'credit_purpose', aggfunc='count')

children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
children_pivot

credit_status,0,1,ratio
number_of_children,,,
1-2 ребенка,6336,647,10.211490
детей нет,13028,1063,8.159349
многодетные,349,31,8.882521


**Вывод** Люди без детей и многодетные отдают кредиты практически одинаково, но всё же бездетные имеют самый лучший показатель возврата кредита. Люди с 1-2 детьми отдают кредиты хуже всего.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [58]:
family_status_pivot = df.pivot_table(index=['family_status'], columns=['credit_status'], values='monthly_income', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot

credit_status,0,1,ratio
family_status,,,
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11408,931,8.160940
не женат / не замужем,2536,274,10.804416


**Вывод** Люди в разводе и вдовы/вдовцы имеют самые лучшие показатели. Граждане, которые не имеют узаконенных отношений, более склонны к просрочке по выплатам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [59]:
monthly_income_pivot = df.pivot_table(index=['income_status'], columns=['credit_status'], values='education_id', aggfunc='count')
monthly_income_pivot['ratio'] = monthly_income_pivot[1] / monthly_income_pivot[0] * 100
monthly_income_pivot

credit_status,0,1,ratio
income_status,,,
Высокий уровень дохода,4882,481,9.852519
Низкий уровень дохода,4945,419,8.473205
Сверхвысокий уровень дохода,4981,383,7.689219
Средний уровень дохода,4905,458,9.337411


Чем больше заработок, тем хуже такие люди выплачивают кредиты (интересная находка, на самом деле).

In [60]:
# Проверим количество людей в каждой из групп
df['income_status'].value_counts(normalize = True)

Сверхвысокий уровень дохода    0.250023
Низкий уровень дохода          0.250023
Высокий уровень дохода         0.249977
Средний уровень дохода         0.249977
Name: income_status, dtype: float64

**Вывод** Люди со сверхвысоким уровнем дохода выплачивают кредиты лучше всего, а вот те, у кого уровень дохода средний и высокий отдают кредиты хуже. Мне не совсем ясно, почему так, но возможно, эти люди просто берут слишком большие суммы, ошибочно полагая, что их заработка хватит на выплаты по кредиты.
Люди с низким уровнем дохода отдают кредиты неплохо: лучше, чем клиенты с высоким и средним уровнями дохода, но хуче, чем клиенты со сверхвысоким уровнем дохода

- Как разные цели кредита влияют на его возврат в срок?

In [61]:
credit_purpose_pivot = df.pivot_table(index=['credit_purpose'], columns=['credit_status'], values='education_id', aggfunc='count')
credit_purpose_pivot['ratio'] = credit_purpose_pivot[1] / credit_purpose_pivot[0] * 100
credit_purpose_pivot

credit_status,0,1,ratio
credit_purpose,,,
автомобиль,3903,403,10.325391
недвижимость,10029,782,7.797388
образование,3643,370,10.156464
свадьба,2138,186,8.699719


**Вывод** 
Заемщики, берущие кредит для приобретения/проведение операций с жильем и на свадьбу, наиболее ответственны и менее склонны нарушать обязательства по выплатам кредита в срок.

## Шаг 4. Общий вывод

Существует зависимость между уровнем дохода, количества детей, целей на кредит и семейным положением. Слишком большой разницы по показателям не было, но возможно, если данных будет больше, то разница станет более очевидной. Это уже может внести существенное влияние на банковскую систему.
Заемщики с официально оформленными отношениями (или которые в прошлом были в официальном в браке) и не имеющие детей - самые ответственные заемщики.Заемщики, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1-2 детей - наименее ответственные заемщики.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.